# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,95.047462,-0.426813,1234,94.620649,95.474274,189.241298,190.948549
6,611.930973,0.404817,1234,612.335790,611.526156,1224.671580,1223.052312
7,416.387189,-0.908138,1234,415.479051,417.295327,830.958102,834.590655
8,178.348333,-0.591225,1234,177.757108,178.939558,355.514215,357.879116
9,101.047487,0.839486,1234,101.886973,100.208001,203.773947,200.416002
10,614.798049,-0.516931,1234,614.281117,615.314980,1228.562235,1230.629960
11,727.530717,0.473764,1234,728.004481,727.056954,1456.008963,1454.113907
12,751.208804,0.165251,1234,751.374055,751.043553,1502.748109,1502.087106
13,866.714087,-0.994355,1234,865.719731,867.708442,1731.439462,1735.416884
14,535.407765,0.923967,1234,536.331731,534.483798,1072.663462,1068.967596


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-94.620649,95.474274
6,-612.335790,611.526156
7,-415.479051,417.295327
8,-177.757108,178.939558
9,-101.886973,100.208001
10,-614.281117,615.314980
11,-728.004481,727.056954
12,-751.374055,751.043553
13,-865.719731,867.708442
14,-536.331731,534.483798


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-94.620649,95.474274
6,-612.335790,611.526156
7,-415.479051,417.295327
8,-177.757108,178.939558
9,-101.886973,100.208001
10,-614.281117,615.314980
11,-728.004481,727.056954
12,-751.374055,751.043553
13,-865.719731,867.708442
14,-536.331731,534.483798


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -94.62064886,   95.47427448],
       [-612.33578976,  611.52615584],
       [-415.47905115,  417.29532735],
       [-177.75710762,  178.93955816],
       [-101.88697326,  100.20800112],
       [-614.28111737,  615.31497978],
       [-728.00448129,  727.05695367],
       [-751.37405455,  751.04355278],
       [-865.71973104,  867.70844203],
       [-536.33173108,  534.483798  ]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,187.241298,-190.948549
6,1222.671580,-1223.052312
7,828.958102,-834.590655
8,353.514215,-357.879116
9,201.773947,-200.416002
10,1226.562235,-1230.629960
11,1454.008963,-1454.113907
12,1500.748109,-1502.087106
13,1729.439462,-1735.416884
14,1070.663462,-1068.967596


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
